In [194]:
# general imports
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import string
import time
import numpy as np

n=6
pi=np.pi

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice, AwsQuantumTask

In [195]:
def run_sim(circuit,i) :
    # set up device
    device = LocalSimulator()
    result = device.run(circuit, shots=Nshots).result()
    inp=result.measurement_counts
    # get id and status of submitted task
    #ionq_task_id = ionq_task.id
    #print('Circuit'+i+' '+ ionq_task_id)
    #ionq_status = ionq_task.state()
    # print('ID of task:', ionq_task_id)
    #print('Status of task:', ionq_status);
    return inp

In [196]:
def g(ind,pauli):
    pauli[(ind+n-1)%n]*=2
    pauli[ind]*=3
    pauli[(ind+n+1)%n]*=2
    return 0
def pauliread(qc,pauli):
    for i in range (n):
        if pauli[i]==3:
            qc.h(i)
        if pauli[i]==4:
            pauli[i]=1
        if pauli[i]==6:
            qc.rz(i,pi/2)
            qc.h(i)
        if pauli[i]==12:
            qc.h(i)
            pauli[n]=-pauli[n]

In [197]:
value=0
for i in range(2**n):
    print(i)
    if(i==0 or i==1 or i==2 or i==4 or i==8 or i==16 or i==32 or i==42 or i==21):
        continue
    pauli=[1]*(n+1)
    t=i
    qc=Circuit()
    for m in range (n):
        qc.h(m)
    for m in range (n):
        qc.h((m+1)%n)
        qc.cnot(control=m,target=(m+1)%n)
        qc.h((m+1)%n)
        
    for j in range (n):
        c=t%(2**(j+1))
        if c!=0:
            g(j,pauli)
        t-=c
    pauliread(qc,pauli)
    #for j in range (n):
    #    if pauli[j]!=0:
    #        qc.measure(j,j)
    inp=run_sim(qc,i)
    #print(inp)
    current_value=0
    for x,y in inp.items():
        #x=x[::-1]
        s=pauli[n]
        for k in range (len(x)):
            if pauli[k]!=0 and pauli[k]!=1 and int(x[k])==1:
                s=-s
        current_value+=s*y/Nshots
    value+=current_value
display(value)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


55.0

In [198]:
### IonQ ###

In [199]:
# Enter the S3 bucket you created during onboarding in the code below
my_bucket = "amazon-braket-289d309751f5" # the name of the bucket
my_prefix = "non-local-game" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)
Nshots = 1024

def run_ionq(circuit,i) :
    # set up device
    ionq = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")

    # run circuit with a polling time of 5 days
    ionq_task = ionq.run(circuit, s3_folder, shots=Nshots, poll_timeout_seconds=5*24*60*60)
    # get id and status of submitted task
    ionq_task_id = ionq_task.id
    print('Circuit'+str(i)+' '+ ionq_task_id)
    ionq_status = ionq_task.state()
    # print('ID of task:', ionq_task_id)
    print('Status of task:', ionq_status);
    return ionq_task_id


In [200]:
N=64 #used to run a smaller number of circuits for testing

taskid=list(range(N))
for i in range(N):
    print(i)
    if(i==0 or i==1 or i==2 or i==4 or i==8 or i==16 or i==32 or i==42 or i==21):
        continue
    pauli=[1]*(n+1)
    t=i
    qc=Circuit()
    for m in range (n):
        qc.h(m)
    for m in range (n):
        qc.h((m+1)%n)
        qc.cnot(control=m,target=(m+1)%n)
        qc.h((m+1)%n)
    for j in range (n):
        c=t%(2**(j+1))
        if c!=0:
            g(j,pauli)
        t-=c
    pauliread(qc,pauli)
    #print(qc)
    
    #### uncomment to run ####
    #taskid[i]=run_ionq(qc,i) 
    
    #print(inp)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [201]:
taskid

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63]

In [207]:
#18/04/2021, 15:33 - 1024 shots - second time
#taskid=[0, 1, 2, 'arn:aws:braket:us-east-1:487494119027:quantum-task/57a44575-1c57-404b-9021-3bf6fe437982', 4, 'arn:aws:braket:us-east-1:487494119027:quantum-task/1f778c20-6f96-45e7-ba7f-d6fa71aa51fb', 'arn:aws:braket:us-east-1:487494119027:quantum-task/7b54bd31-5da3-4050-ae4d-1ac463423685', 'arn:aws:braket:us-east-1:487494119027:quantum-task/1737ba5e-a9c1-438c-b3c9-45941e7283d9', 8, 'arn:aws:braket:us-east-1:487494119027:quantum-task/68b0ce65-56b4-460f-84a0-e9d05e2bf86e', 'arn:aws:braket:us-east-1:487494119027:quantum-task/2fcfeaf4-db6d-42c4-b7db-d081a5ce3095', 'arn:aws:braket:us-east-1:487494119027:quantum-task/00b9c1fe-38c6-48ac-b26c-dcb2118c36e6', 'arn:aws:braket:us-east-1:487494119027:quantum-task/8fd45de5-14b2-484b-bc69-e4cc707357c3', 'arn:aws:braket:us-east-1:487494119027:quantum-task/cb3fcd8f-6c64-4649-9b03-094a659d1202', 'arn:aws:braket:us-east-1:487494119027:quantum-task/5a7164e6-763d-48ff-8de7-135e830b7a20', 'arn:aws:braket:us-east-1:487494119027:quantum-task/33fad0cd-6fe8-445d-af3a-08d34c05612d', 16, 'arn:aws:braket:us-east-1:487494119027:quantum-task/fc4e1187-cfcb-479d-a368-b3f78cf32e5c', 'arn:aws:braket:us-east-1:487494119027:quantum-task/2d11a173-cf53-4d60-b61e-0db61b49b113', 'arn:aws:braket:us-east-1:487494119027:quantum-task/80178173-dbec-451f-8975-e99b03570213', 'arn:aws:braket:us-east-1:487494119027:quantum-task/9425bb10-b900-48bc-91f3-b9a49dfa7ec1', 21, 'arn:aws:braket:us-east-1:487494119027:quantum-task/1e5cf142-cbb9-475f-be6a-b3386c5079d4', 'arn:aws:braket:us-east-1:487494119027:quantum-task/0cb17c1d-2137-4211-be3d-7617eb855946', 'arn:aws:braket:us-east-1:487494119027:quantum-task/69dd6edf-6604-41b4-a05b-1d1d0f0768e1', 'arn:aws:braket:us-east-1:487494119027:quantum-task/558335a3-a51e-4efd-8c7b-ea92f018d8cf', 'arn:aws:braket:us-east-1:487494119027:quantum-task/5489bcc3-d6e7-4d01-829b-7ee05e549fe1', 'arn:aws:braket:us-east-1:487494119027:quantum-task/9f49a8a8-5e45-49c3-9da8-7da3267155f9', 'arn:aws:braket:us-east-1:487494119027:quantum-task/77adcb32-829a-4f52-9bd7-f50fc038582b', 'arn:aws:braket:us-east-1:487494119027:quantum-task/248e8127-17d1-46f8-ac6e-62f54b33e568', 'arn:aws:braket:us-east-1:487494119027:quantum-task/c93d0948-fd05-4201-b55b-3765ea3b3683', 'arn:aws:braket:us-east-1:487494119027:quantum-task/59a90447-860a-4cbe-9530-fa2c23f4c822', 32, 'arn:aws:braket:us-east-1:487494119027:quantum-task/040235e8-a0b1-4056-a0a9-a47c54e5dd7d', 'arn:aws:braket:us-east-1:487494119027:quantum-task/ac484218-8437-4481-b8e2-56f055154b79', 'arn:aws:braket:us-east-1:487494119027:quantum-task/c38d9ccf-1947-474d-b72c-ddd2ab9b03f2', 'arn:aws:braket:us-east-1:487494119027:quantum-task/312df5d8-0a45-4559-90d5-e69b86343144', 'arn:aws:braket:us-east-1:487494119027:quantum-task/045de985-b719-48c1-a8b0-5a0f16b7ac31', 'arn:aws:braket:us-east-1:487494119027:quantum-task/2aa8e819-1b24-452a-afad-6724c8bfd6c9', 'arn:aws:braket:us-east-1:487494119027:quantum-task/25434998-d8e2-4952-848b-29b2fed7f65d', 'arn:aws:braket:us-east-1:487494119027:quantum-task/6cdd26ed-5dde-47e3-801c-180651252407', 'arn:aws:braket:us-east-1:487494119027:quantum-task/08a3db07-7443-48ba-a518-f598d2ac3354', 42, 'arn:aws:braket:us-east-1:487494119027:quantum-task/f25d1ded-b3ca-4554-87a5-51cb38929391', 'arn:aws:braket:us-east-1:487494119027:quantum-task/8e9313fc-38f4-4b22-8e3e-54925b506583', 'arn:aws:braket:us-east-1:487494119027:quantum-task/1816b3d5-daa8-4026-b892-edee4e5c1989', 'arn:aws:braket:us-east-1:487494119027:quantum-task/d370f452-33fc-4c46-800d-53b37bd60fdd', 'arn:aws:braket:us-east-1:487494119027:quantum-task/9b5f0b22-e713-4ba3-8966-3d7a16e4e5ae', 'arn:aws:braket:us-east-1:487494119027:quantum-task/a8496e48-170d-4f11-870a-ee3dd49344bf', 'arn:aws:braket:us-east-1:487494119027:quantum-task/970ac2a4-0fc5-4598-ae46-f633179b44fc', 'arn:aws:braket:us-east-1:487494119027:quantum-task/3b23af1c-da82-4051-a4a9-df6cc70d0f87', 'arn:aws:braket:us-east-1:487494119027:quantum-task/4cea7102-46eb-408d-9439-9bf62f2e62c8', 'arn:aws:braket:us-east-1:487494119027:quantum-task/63ad67dc-03f1-43f6-aacd-1153b38c74be', 'arn:aws:braket:us-east-1:487494119027:quantum-task/c37751bc-46dc-4fc3-b0d9-74fe7403732c', 'arn:aws:braket:us-east-1:487494119027:quantum-task/94e3dd5d-39db-4320-9f1f-63cf8b947a21', 'arn:aws:braket:us-east-1:487494119027:quantum-task/7034912c-73aa-416f-b7f6-4ff8c7834eff', 'arn:aws:braket:us-east-1:487494119027:quantum-task/5c992ed3-4410-4d83-8658-e179b3239737', 'arn:aws:braket:us-east-1:487494119027:quantum-task/d0184034-c8e9-4a2b-b5df-baf33d5bb1fe', 'arn:aws:braket:us-east-1:487494119027:quantum-task/e52fcd29-d872-42a9-894f-5bfceb52e20a', 'arn:aws:braket:us-east-1:487494119027:quantum-task/8b93b05a-de15-4a29-b02f-9f24b93967d2', 'arn:aws:braket:us-east-1:487494119027:quantum-task/fcf36c31-a971-40dd-8d56-b06c904ce9a6', 'arn:aws:braket:us-east-1:487494119027:quantum-task/b84f8a49-eb80-4e13-81f7-d684eab7463b', 'arn:aws:braket:us-east-1:487494119027:quantum-task/acfa51cb-69b2-4b8e-b238-0a3d76c33ffd', 'arn:aws:braket:us-east-1:487494119027:quantum-task/f644b127-5870-4522-a98c-546b2c1cc713']

#18/04/2021, 11:10 - 1024 shots
taskid = [0, 1, 2, 'arn:aws:braket:us-east-1:487494119027:quantum-task/db6645ae-3c35-4dce-bcec-3a32d885f282', 4, 'arn:aws:braket:us-east-1:487494119027:quantum-task/66205cd8-c9ef-4f92-a273-127d00fe527c', 'arn:aws:braket:us-east-1:487494119027:quantum-task/d1ae3d9c-8cb3-49d0-898b-415aa4f252cd', 'arn:aws:braket:us-east-1:487494119027:quantum-task/f21ee517-9efd-423a-b176-cb57b1bf3c7e', 8, 'arn:aws:braket:us-east-1:487494119027:quantum-task/a9e2ca42-12bc-4143-8f91-f4c24114748e', 'arn:aws:braket:us-east-1:487494119027:quantum-task/29d47d8e-63b0-426c-baa4-02da3ebf8088', 'arn:aws:braket:us-east-1:487494119027:quantum-task/dd4a95fe-cc50-4e4b-9575-5096f18ce536', 'arn:aws:braket:us-east-1:487494119027:quantum-task/135cca5f-6ea9-4c15-8431-ee992e58fd1a', 'arn:aws:braket:us-east-1:487494119027:quantum-task/9552cc5c-7b62-4723-aceb-8f03db366a28', 'arn:aws:braket:us-east-1:487494119027:quantum-task/6c6ace7a-4ea9-44fb-96ac-1094affe06d7', 'arn:aws:braket:us-east-1:487494119027:quantum-task/5481c883-b4f6-40bf-91ed-91ea5d10bbc9', 16, 'arn:aws:braket:us-east-1:487494119027:quantum-task/8ad372f8-4faa-466b-8c2d-55e89728dcbb', 'arn:aws:braket:us-east-1:487494119027:quantum-task/0ba379b7-c89d-479e-9686-3d696f8a0eee', 'arn:aws:braket:us-east-1:487494119027:quantum-task/f71ffe48-865e-467f-a2b4-7ab37cd1dc7b', 'arn:aws:braket:us-east-1:487494119027:quantum-task/3d054a59-6a74-4910-9ab5-ac3dba5f5f12', 21, 'arn:aws:braket:us-east-1:487494119027:quantum-task/09ceea73-9595-4db6-ab1a-82e0a3c73ebd', 'arn:aws:braket:us-east-1:487494119027:quantum-task/4f1be777-b8a3-4da7-a515-d6e405ee55df', 'arn:aws:braket:us-east-1:487494119027:quantum-task/b44f7061-1e18-47dc-a1e1-7935eb293a42', 'arn:aws:braket:us-east-1:487494119027:quantum-task/888013f9-34b6-4932-a66c-4356837cf0da', 'arn:aws:braket:us-east-1:487494119027:quantum-task/f5ce85a1-e056-4982-801d-a7c297e0d620', 'arn:aws:braket:us-east-1:487494119027:quantum-task/0e42874f-2676-4e2d-9dc7-6c3cc17ffc0c', 'arn:aws:braket:us-east-1:487494119027:quantum-task/7d102816-e02e-4580-9642-d4a4fc74c7c9', 'arn:aws:braket:us-east-1:487494119027:quantum-task/d003fff7-a811-49f3-a803-7aee29e05cf8', 'arn:aws:braket:us-east-1:487494119027:quantum-task/ec415ad2-b8b0-4dcb-8683-fad32b4cd9da', 'arn:aws:braket:us-east-1:487494119027:quantum-task/c3e4e6ae-0c95-4601-9341-bc1e33a284f1', 32, 'arn:aws:braket:us-east-1:487494119027:quantum-task/932dc236-8b5a-4ab3-a014-ab96a9cbe4c4', 'arn:aws:braket:us-east-1:487494119027:quantum-task/51c25f1f-3617-4469-ad8f-174c539ca45e', 'arn:aws:braket:us-east-1:487494119027:quantum-task/9a2b2bb2-7c24-4572-84fe-724cb33ba902', 'arn:aws:braket:us-east-1:487494119027:quantum-task/bd58fe56-4f6d-4481-b032-45470b7ef575', 'arn:aws:braket:us-east-1:487494119027:quantum-task/8e526f87-bcb0-4e57-b40e-d0f4136ee834', 'arn:aws:braket:us-east-1:487494119027:quantum-task/e9312276-04e9-4877-b4e0-4cbdd8fa7d26', 'arn:aws:braket:us-east-1:487494119027:quantum-task/6b4463ac-e974-4ba1-9182-2882a788a0fe', 'arn:aws:braket:us-east-1:487494119027:quantum-task/06abeacc-f665-4e4e-9fff-753ed6069494', 'arn:aws:braket:us-east-1:487494119027:quantum-task/fe011e55-fa9e-4cb3-b57d-57384d5ac9c5', 42, 'arn:aws:braket:us-east-1:487494119027:quantum-task/8232d4ea-170d-4c51-89eb-a2dd4661e960', 'arn:aws:braket:us-east-1:487494119027:quantum-task/63484a95-ff11-411a-95e0-1f5a0a9459f6', 'arn:aws:braket:us-east-1:487494119027:quantum-task/f8764fcb-b52a-4baa-bd20-2d3425b7031c', 'arn:aws:braket:us-east-1:487494119027:quantum-task/77897e15-cb4e-41b9-a8eb-18e9166bb2e7', 'arn:aws:braket:us-east-1:487494119027:quantum-task/1ce55917-eee1-4bb5-b92f-ad9c19e7793d', 'arn:aws:braket:us-east-1:487494119027:quantum-task/f66ab4eb-b5de-4520-9178-1a699ba485a4', 'arn:aws:braket:us-east-1:487494119027:quantum-task/1c885b6d-cb60-4bd5-ba19-6ea0314f7dc9', 'arn:aws:braket:us-east-1:487494119027:quantum-task/77e42199-833f-4bb5-a997-f6ed213a3052', 'arn:aws:braket:us-east-1:487494119027:quantum-task/cfaffca9-7beb-4339-bbcc-0381e0310379', 'arn:aws:braket:us-east-1:487494119027:quantum-task/6846d8de-91da-4c5e-b4b2-1024cad06dc6', 'arn:aws:braket:us-east-1:487494119027:quantum-task/8536b482-b398-4bea-92ab-a38a82cd4ef5', 'arn:aws:braket:us-east-1:487494119027:quantum-task/d6136d11-62e6-4607-a863-f4bb920ea2d2', 'arn:aws:braket:us-east-1:487494119027:quantum-task/2dca92c0-9843-4504-b64c-60b1ee1779b5', 'arn:aws:braket:us-east-1:487494119027:quantum-task/20caaaaf-8cc0-449f-bb7a-5889efe37500', 'arn:aws:braket:us-east-1:487494119027:quantum-task/23cfec65-5639-4e35-9bcc-fdc0bdcc7906', 'arn:aws:braket:us-east-1:487494119027:quantum-task/cc39f415-43ed-4205-8b05-485a9507e3d6', 'arn:aws:braket:us-east-1:487494119027:quantum-task/47cf0639-55eb-4910-b1c8-23003d68c655', 'arn:aws:braket:us-east-1:487494119027:quantum-task/761de390-2a99-410c-bfba-16764037c0d6', 'arn:aws:braket:us-east-1:487494119027:quantum-task/8f7e64d0-0ee1-473f-aa74-c9d345914750', 'arn:aws:braket:us-east-1:487494119027:quantum-task/c6f6bc9c-6a23-4952-9f48-3584269a6b4f', 'arn:aws:braket:us-east-1:487494119027:quantum-task/528c5bf1-f6aa-4904-a338-2153dfdb9cf6']



In [203]:
for i in range(N) :
    if len(str(taskid[i]))>2 :
        task = AwsQuantumTask(arn=taskid[i])
        print(str(i)+" "+task.state())
        #task.cancel()

3 COMPLETED
5 COMPLETED
6 COMPLETED
7 COMPLETED
9 COMPLETED
10 COMPLETED
11 COMPLETED
12 COMPLETED
13 COMPLETED
14 COMPLETED
15 COMPLETED
17 COMPLETED
18 COMPLETED
19 COMPLETED
20 COMPLETED
22 COMPLETED
23 COMPLETED
24 COMPLETED
25 COMPLETED
26 COMPLETED
27 COMPLETED
28 COMPLETED
29 COMPLETED
30 COMPLETED
31 COMPLETED
33 COMPLETED
34 COMPLETED
35 COMPLETED
36 COMPLETED
37 COMPLETED
38 COMPLETED
39 COMPLETED
40 COMPLETED
41 COMPLETED
43 COMPLETED
44 COMPLETED
45 COMPLETED
46 COMPLETED
47 COMPLETED
48 COMPLETED
49 COMPLETED
50 COMPLETED
51 COMPLETED
52 COMPLETED
53 COMPLETED
54 COMPLETED
55 COMPLETED
56 COMPLETED
57 COMPLETED
58 COMPLETED
59 COMPLETED
60 COMPLETED
61 COMPLETED
62 COMPLETED
63 COMPLETED


In [205]:
import json
import boto3

def getcounts(taskid):
    s3 = boto3.client('s3')
    my_key=my_prefix+"/"+taskid+"/results.json"
    #print(my_bucket)
    #print(my_key)
    data = s3.get_object(Bucket=my_bucket, Key=my_key)
    json_data = data['Body'].read().decode('utf-8')
    json_content = json.loads(json_data)
    return json_content['measurementProbabilities']

In [209]:
value=0
N=64
for i in range(N):
    #print(i)
    if(i==0 or i==1 or i==2 or i==4 or i==8 or i==16 or i==32 or i==42 or i==21):
        continue
    pauli=[1]*(n+1)
    t=i
    qc=Circuit()
    for m in range (n):
        qc.h(m)
    for m in range (n):
        qc.h((m+1)%n)
        qc.cnot(control=m,target=(m+1)%n)
        qc.h((m+1)%n)
        
    for j in range (n):
        c=t%(2**(j+1))
        if c!=0:
            g(j,pauli)
        t-=c
    pauliread(qc,pauli)
    
    shortid=taskid[i][51:]
    inp=getcounts(shortid)
    #print(inp)
    current_value=0
    for x,y in inp.items():
        #x=x[::-1]
        s=pauli[n]
        for k in range (len(x)):
            if pauli[k]!=1 and int(x[k])==1:
                s=-s
        #print(x,s,y)
        current_value+=s*y
    print(current_value)
    value+=current_value
display(value)/55

0.8495999999999998
0.7284999999999997
0.7088999999999995
0.7690999999999996
0.7281999999999998
0.7384999999999997
0.8106999999999998
0.8281999999999998
0.7185999999999996
0.6893999999999998
0.7240999999999997
0.6932999999999998
0.6857000000000001
0.7497999999999999
0.838
0.6524999999999997
0.787
0.8158999999999997
0.6739999999999999
0.7248999999999999
0.7517999999999996
0.8277999999999998
0.6546999999999998
0.6249999999999997
0.7538999999999995
0.7874999999999994
0.7656999999999998
0.8119999999999997
0.7555
0.7091999999999999
0.7029999999999998
0.7500999999999997
0.8412
0.7420999999999998
0.8084999999999998
0.7866
0.6873999999999998
0.6872999999999998
0.707
0.8669999999999995
0.7481999999999995
0.7733999999999999
0.8002999999999996
0.8182999999999998
0.7011999999999998
0.7263000000000001
0.7267999999999996
0.7765999999999995
0.7595999999999996
0.6757
0.7300000000000003
0.7360999999999998
0.6643999999999999
0.6638999999999996
0.7381999999999999


40.97719999999998

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [210]:
display(value)

40.97719999999998

In [212]:
inp.items()

dict_items([('000000', 0.0576), ('100000', 0.001), ('010000', 0.0049), ('110000', 0.0029), ('001000', 0.0039), ('101000', 0.0674), ('011000', 0.0088), ('111000', 0.0059), ('000100', 0.002), ('100100', 0.0059), ('010100', 0.0449), ('110100', 0.0049), ('001100', 0.0029), ('101100', 0.0049), ('011100', 0.001), ('111100', 0.0488), ('000010', 0.0059), ('100010', 0.0479), ('010010', 0.002), ('110010', 0.0068), ('001010', 0.0342), ('101010', 0.0039), ('011010', 0.0029), ('111010', 0.0039), ('000110', 0.0068), ('100110', 0.0039), ('010110', 0.0039), ('110110', 0.0469), ('001110', 0.0049), ('101110', 0.002), ('011110', 0.0537), ('111110', 0.0029), ('000001', 0.0068), ('100001', 0.0039), ('010001', 0.0527), ('110001', 0.0068), ('001001', 0.002), ('101001', 0.0049), ('011001', 0.0029), ('111001', 0.0537), ('000101', 0.0518), ('100101', 0.002), ('010101', 0.0039), ('110101', 0.001), ('001101', 0.002), ('101101', 0.0381), ('011101', 0.0029), ('111101', 0.0059), ('000011', 0.0039), ('100011', 0.0029

In [213]:
Nshots

1024

In [214]:
0.0088*1024

9.0112